## theScore Data Scraper

The following code snippets allow us to gather betting line data at closing for NBA games.

In [1]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [47]:
# constants that affect scrapage
baseURL = 'https://www.thescore.com/nba/events/'
most_recent_game_id = 508062
earliest_game_id = 507514
# There will be a total of 1080 games this season, 30*72/2; 1230 in a normal season
num_games = most_recent_game_id - earliest_game_id

In [51]:

response_data = []
for game_id in range(most_recent_game_id, most_recent_game_id - num_games, -1):
    page = requests.get(baseURL + str(game_id))
    soup = BeautifulSoup(page.content, 'html.parser')
    # only true if the game has already finished
    label_text = soup.find_all('div', {'class': 'GameDetailsCard__label--iBMhJ'})
    if not label_text:
        continue
    has_closing_odds = label_text[len(label_text) - 1].contents[0] == 'Closing Odds:'
    if has_closing_odds:
        team_data = soup.find_all('div', {'class': 'Matchup__teamName--vqpde'})
        if not team_data:
            continue
        team_1_abbrv = team_data[0].contents[0].partition(' ')[0]
        team_2_abbrv = team_data[1].contents[0].partition(' ')[0]
        odds = soup.find_all('div', {'class': 'GameDetailsCard__content--2L_KF'})
        if len(odds) > 0:
            odds_string = odds[len(odds) - 1].contents[0]
            odds_partition = odds_string.split()
            if len(odds_partition) < 4:
                continue
            favorite = odds_partition[0]
            spread = odds_partition[1][0:len(odds_partition[1]) - 1]
            ou = odds_partition[3]
            scores = soup.find_all('div', {'class': 'Matchup__teamScore--2BeCA'})
            if not scores:
                continue
            team_1_score = scores[0].contents[0]
            team_2_score = scores[1].contents[0]
            
            print(game_id)
            stats_page = requests.get(baseURL+str(game_id)+'/stats')
            soup = BeautifulSoup(stats_page.content, 'html.parser')
            players = soup.find_all('div', {'class': 'BoxScore__boxScore--tDnlB'})#BoxScore__statLine--3Daky
            stats = []
            for i in range(0,len(players)):
                for j in range(1, 15):
                    stats.append(players[i].contents[j].contents[0].contents[0].contents[0].contents[0])
                    for k in range(1, len(players[i].contents[j].contents[0].contents[0])):
                        if players[i].contents[j].contents[0].contents[0].contents[k].contents[0].isnumeric():
                            stats.append(int(players[i].contents[j].contents[0].contents[0].contents[k].contents[0]))
                        else:    
                            stats.append(players[i].contents[j].contents[0].contents[0].contents[k].contents[0])
            stats
                        
            responses = [team_1_abbrv, 
                                  team_2_abbrv,
                                  favorite,
                                  spread,
                                  ou,
                                  team_1_score,
                                  team_2_score]
            
            for i in stats:
                responses.append(i)
            response_data.append(responses)


df = pd.DataFrame(response_data,
                  columns = ['team1', 
                             'team2', 
                             'favorite', 
                             'spread', 
                             'over_under', 
                             'team1_score',
                             'team2_score', 'team1p1', 'p1min', 'p1pts', 'p1reb', 'p1ast', 'p1stl', 'p1blk', 'p1pf', 'p1to', 'p1oreb', 'p1dreb', 'p1fg', 'p1fg%', 'p1ft', 'p1ft%', 'p13pt', 'p13pt%', 'p1+/-','team1p2', 'p2min', 'p2pts', 'p2reb', 'p2ast', 'p2stl', 'p2blk', 'p2pf', 'p2to', 'p2oreb', 'p2dreb', 'p2fg', 'p2fg%', 'p2ft', 'p2ft%', 'p23pt', 'p23pt%', 'p2+/-','team1p3', 'p3min', 'p3pts', 'p3reb', 'p3ast', 'p3stl', 'p3blk', 'p3pf', 'p3to', 'p3oreb', 'p3dreb', 'p3fg', 'p3fg%', 'p3ft', 'p3ft%', 'p33pt', 'p33pt%', 'p3+/-','team1p4', 'p4min', 'p4pts', 'p4reb', 'p4ast', 'p4stl', 'p4blk', 'p4pf', 'p4to', 'p4oreb', 'p4dreb', 'p4fg', 'p4fg%', 'p4ft', 'p4ft%', 'p43pt', 'p43pt%', 'p4+/-','team1p5', 'p5min', 'p5pts', 'p5reb', 'p5ast', 'p5stl', 'p5blk', 'p5pf', 'p5to', 'p5oreb', 'p5dreb', 'p5fg', 'p5fg%', 'p5ft', 'p5ft%', 'p53pt', 'p53pt%', 'p5+/-','team1p6', 'p6min', 'p6pts', 'p6reb', 'p6ast', 'p6stl', 'p6blk', 'p6pf', 'p6to', 'p6oreb', 'p6dreb', 'p6fg', 'p6fg%', 'p6ft', 'p6ft%', 'p63pt', 'p63pt%', 'p6+/-','team1p7', 'p7min', 'p7pts', 'p7reb', 'p7ast', 'p7stl', 'p7blk', 'p7pf', 'p7to', 'p7oreb', 'p7dreb', 'p7fg', 'p7fg%', 'p7ft', 'p7ft%', 'p73pt', 'p73pt%', 'p7+/-','team1p8', 'p8min', 'p8pts', 'p8reb', 'p8ast', 'p8stl', 'p8blk', 'p8pf', 'p8to', 'p8oreb', 'p8dreb', 'p8fg', 'p8fg%', 'p8ft', 'p8ft%', 'p83pt', 'p83pt%', 'p8+/-','team1p9', 'p9min', 'p9pts', 'p9reb', 'p9ast', 'p9stl', 'p9blk', 'p9pf', 'p9to', 'p9oreb', 'p9dreb', 'p9fg', 'p9fg%', 'p9ft', 'p9ft%', 'p93pt', 'p93pt%', 'p9+/-','team1p10', 'p10min', 'p10pts', 'p10reb', 'p10ast', 'p10stl', 'p10blk', 'p10pf', 'p10to', 'p10oreb', 'p10dreb', 'p10fg', 'p10fg%', 'p10ft', 'p10ft%', 'p103pt', 'p103pt%', 'p10+/-','team1p11', 'p11min', 'p11pts', 'p11reb', 'p11ast', 'p11stl', 'p11blk', 'p11pf', 'p11to', 'p11oreb', 'p11dreb', 'p11fg', 'p11fg%', 'p11ft', 'p11ft%', 'p113pt', 'p113pt%', 'p11+/-','team1p12', 'p12min', 'p12pts', 'p12reb', 'p12ast', 'p12stl', 'p12blk', 'p12pf', 'p12to', 'p12oreb', 'p12dreb', 'p12fg', 'p12fg%', 'p12ft', 'p12ft%', 'p123pt', 'p123pt%', 'p12+/-','team1p13', 'p13min', 'p13pts', 'p13reb', 'p13ast', 'p13stl', 'p13blk', 'p13pf', 'p13to', 'p13oreb', 'p13dreb', 'p13fg', 'p13fg%', 'p13ft', 'p13ft%', 'p133pt', 'p133pt%', 'p13+/-','team1p14', 'p14min', 'p14pts', 'p14reb', 'p14ast', 'p14stl', 'p14blk', 'p14pf', 'p14to', 'p14oreb', 'p14dreb', 'p14fg', 'p14fg%', 'p14ft', 'p14ft%', 'p143pt', 'p143pt%', 'p14+/-','team2p1', 'p1min', 'p1pts', 'p1reb', 'p1ast', 'p1stl', 'p1blk', 'p1pf', 'p1to', 'p1oreb', 'p1dreb', 'p1fg', 'p1fg%', 'p1ft', 'p1ft%', 'p13pt', 'p13pt%', 'p1+/-','team2p2', 'p2min', 'p2pts', 'p2reb', 'p2ast', 'p2stl', 'p2blk', 'p2pf', 'p2to', 'p2oreb', 'p2dreb', 'p2fg', 'p2fg%', 'p2ft', 'p2ft%', 'p23pt', 'p23pt%', 'p2+/-','team2p3', 'p3min', 'p3pts', 'p3reb', 'p3ast', 'p3stl', 'p3blk', 'p3pf', 'p3to', 'p3oreb', 'p3dreb', 'p3fg', 'p3fg%', 'p3ft', 'p3ft%', 'p33pt', 'p33pt%', 'p3+/-','team2p4', 'p4min', 'p4pts', 'p4reb', 'p4ast', 'p4stl', 'p4blk', 'p4pf', 'p4to', 'p4oreb', 'p4dreb', 'p4fg', 'p4fg%', 'p4ft', 'p4ft%', 'p43pt', 'p43pt%', 'p4+/-','team2p5', 'p5min', 'p5pts', 'p5reb', 'p5ast', 'p5stl', 'p5blk', 'p5pf', 'p5to', 'p5oreb', 'p5dreb', 'p5fg', 'p5fg%', 'p5ft', 'p5ft%', 'p53pt', 'p53pt%', 'p5+/-','team2p6', 'p6min', 'p6pts', 'p6reb', 'p6ast', 'p6stl', 'p6blk', 'p6pf', 'p6to', 'p6oreb', 'p6dreb', 'p6fg', 'p6fg%', 'p6ft', 'p6ft%', 'p63pt', 'p63pt%', 'p6+/-','team2p7', 'p7min', 'p7pts', 'p7reb', 'p7ast', 'p7stl', 'p7blk', 'p7pf', 'p7to', 'p7oreb', 'p7dreb', 'p7fg', 'p7fg%', 'p7ft', 'p7ft%', 'p73pt', 'p73pt%', 'p7+/-','team2p8', 'p8min', 'p8pts', 'p8reb', 'p8ast', 'p8stl', 'p8blk', 'p8pf', 'p8to', 'p8oreb', 'p8dreb', 'p8fg', 'p8fg%', 'p8ft', 'p8ft%', 'p83pt', 'p83pt%', 'p8+/-','team2p9', 'p9min', 'p9pts', 'p9reb', 'p9ast', 'p9stl', 'p9blk', 'p9pf', 'p9to', 'p9oreb', 'p9dreb', 'p9fg', 'p9fg%', 'p9ft', 'p9ft%', 'p93pt', 'p93pt%', 'p9+/-','team2p10', 'p10min', 'p10pts', 'p10reb', 'p10ast', 'p10stl', 'p10blk', 'p10pf', 'p10to', 'p10oreb', 'p10dreb', 'p10fg', 'p10fg%', 'p10ft', 'p10ft%', 'p103pt', 'p103pt%', 'p10+/-','team2p11', 'p11min', 'p11pts', 'p11reb', 'p11ast', 'p11stl', 'p11blk', 'p11pf', 'p11to', 'p11oreb', 'p11dreb', 'p11fg', 'p11fg%', 'p11ft', 'p11ft%', 'p113pt', 'p113pt%', 'p11+/-','team2p12', 'p12min', 'p12pts', 'p12reb', 'p12ast', 'p12stl', 'p12blk', 'p12pf', 'p12to', 'p12oreb', 'p12dreb', 'p12fg', 'p12fg%', 'p12ft', 'p12ft%', 'p123pt', 'p123pt%', 'p12+/-','team2p13', 'p13min', 'p13pts', 'p13reb', 'p13ast', 'p13stl', 'p13blk', 'p13pf', 'p13to', 'p13oreb', 'p13dreb', 'p13fg', 'p13fg%', 'p13ft', 'p13ft%', 'p133pt', 'p133pt%', 'p13+/-','team2p14', 'p14min', 'p14pts', 'p14reb', 'p14ast', 'p14stl', 'p14blk', 'p14pf', 'p14to', 'p14oreb', 'p14dreb', 'p14fg', 'p14fg%', 'p14ft', 'p14ft%', 'p143pt', 'p143pt%', 'p14+/-'])
df.to_csv('lines.csv', index = False)
print(df)

508062
508061
508060
508059
508058
508057
508056
508055
508054
508053
508052
508051
508050
508049
508048
508047
508046
508045
508044
508043
508042
508040
508039
508038
508037
508036
508035
508034
508033
508032
508031
508030
508029
508028
508027
508025
508024
508023
508022
508021
508020
508019
508018
508017
508016
508015
508014
508013
508012
508011
508010
508009
508008
508007
508006
508005
508004
508003
508002
508001
508000
507999
507998
507997
507996
507995
507994
507993
507992
507991
507990
507989
507988
507987
507986
507985
507984
507983
507982
507981
507980
507979
507978
507976
507975
507974
507973
507972
507971
507970
507969
507968
507967
507966
507965
507963
507961
507959
507958
507957
507956
507955
507954
507953
507951
507950
507949
507948
507947
507945
507944
507943
507942
507941
507939
507937
507936
507935
507934
507933
507932
507930
507929
507928
507927
507926
507925
507924
507923
507922
507921
507920
507919
507918
507917
507916
507915
507914
507913
507912
507911
507910
507909